In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [8]:
with ZipFile('data2.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('final.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [9]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [10]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,4909294510,,,нет . . . у тебя на лице написано - нет .,0,тогда я попытался с двумя другими женщинами и ...
1,4909294510,,,нет . . . у тебя на лице написано - нет .,1,"я улыбаюсь потому . . . потому что , описывая ..."
2,4909294510,,,нет . . . у тебя на лице написано - нет .,2,"это , так сказать , соответствует уровню моей ..."
3,4909294510,,,нет . . . у тебя на лице написано - нет .,3,я врач .
4,4909294510,,,нет . . . у тебя на лице написано - нет .,4,не обращайте на меня внимания !


In [11]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [12]:
pd_public.count()

context_id    104834
context_2     104834
context_1     104834
context_0     104834
reply_id      104834
reply         104834
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [10]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (9968, 6), (107501, 8))

In [11]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.092725
 0.0    0.323439
 1.0    0.104687
 2.0    0.479149
Name: context_id, dtype: float64

# Простой бейзлайн

## features

In [12]:
from scipy import sparse

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv_all = CountVectorizer()
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]].apply(lambda x: " ".join(x), axis=1)
cv_all.fit(s_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [35]:
def to_features(df):
    context = cv_all.transform(df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1))
    reply = cv_all.transform(df["reply"])
    features = sparse.hstack([context, reply])
    return features

## crossval

In [16]:
from tqdm import tqdm_notebook

In [17]:
from sklearn.model_selection import StratifiedKFold

In [18]:
skf = StratifiedKFold(n_splits=3)

In [19]:
from sklearn.linear_model import LinearRegression

In [32]:
class RandomMinMaxRegressor(object):
    def __init__(self):
        self.min_y = None
        self.max_y = None
    
    def fit(self, x, y):
        self.min_y = y.min()
        self.max_y = y.max()
    
    def predict(self, x):
        return np.random.random(x.shape[0]) * (self.max_y - self.min_y) + self.min_y        

In [ ]:
model_reg = LinearRegression()

In [33]:
model_reg = RandomMinMaxRegressor()

In [20]:
from sklearn.metrics import mean_absolute_error

In [21]:
def calc_ndcg_score(df, y_predict):
    df = df.copy()
    df["predict"] = y_predict
    #dcg
    df.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["dcg"] = df["label"] / np.log2(df["row_count"] + 1)
    #idcg
    df.sort_values(["context_id", "label"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["idcg"] = df["label"] / np.log2(df["row_count"] + 1)
    
    df_context = df.groupby("context_id").sum()
    return (df_context["dcg"] / df_context["idcg"]).mean() * 100000

In [36]:
def do_cross_val(model_reg):
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_train = pd_train.iloc[_train]
        i_test = pd_train.iloc[_test]

        X_train = to_features(i_train)
        X_test = to_features(i_test)
        Y_train = i_train.label.values
        Y_test = i_test.label.values

        model_reg.fit(X_train, Y_train)
        Y_predict = model_reg.predict(X_test)
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)

In [39]:
do_cross_val(LinearRegression())

A Jupyter Widget


[1.4865290758926417, 1.4505487927986205, 1.5525054989445051]
[84992.04634993454, 85093.63778087734, 85223.37180947387]


In [38]:
do_cross_val(RandomMinMaxRegressor())

A Jupyter Widget


[0.94215100562127641, 0.94450351137377053, 0.94044066978111984]
[82635.87988159526, 82487.80155424896, 82826.7969582632]


In [49]:
def do_worst_ndcg():
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_test = pd_train.iloc[_test].copy().sort_values(["context_id", "label"], ascending=[True, False])
        
        Y_test = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        Y_predict = i_test.sort_values(["context_id", "label"], ascending=[True, True]).label.values
        
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)
do_worst_ndcg()

A Jupyter Widget


[1.1195250984251968, 1.1126695579957553, 1.1291294986158105]
[77475.71260547417, 77562.26032466823, 77472.45724727474]


In [48]:
def do_best_ndcg():
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_test = pd_train.iloc[_test].copy().sort_values(["context_id", "label"], ascending=[True, False])
        
        Y_test = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        Y_predict = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)
do_best_ndcg()

A Jupyter Widget


[0.0, 0.0, 0.0]
[100000.0, 100000.0, 100000.0]


# Submit

In [ ]:
model_reg = None

In [117]:
i_train = pd_train
i_test = pd_public.copy()

X_train = to_features(i_train)
X_test = to_features(i_test)
Y_train = i_train.label.values

model_reg.fit(X_train, Y_train)
Y_predict = model_reg.predict(X_test)

features.shape = (97533, 86012)
features.shape = (9968, 86012)


In [118]:
i_test["predict"] = Y_predict
i_test.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)

In [121]:
i_test.head(15)

,context_id,context_2,context_1,context_0,reply_id,reply,predict
3477,100097508986637,,,выключай их,2,выключить ?,2.370890
3475,100097508986637,,,выключай их,0,выключить что ?,2.365175
3479,100097508986637,,,выключай их,4,"пытаюсь , сэр .",2.003706
3478,100097508986637,,,выключай их,3,я пытаюсь !,1.684137
3480,100097508986637,,,выключай их,5,выключите,1.299290
3476,100097508986637,,,выключай их,1,"ты не выключишь , тогда € сам выключу",-0.009512
3483,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,2,я не могу позволить вам уйти .,2.107446
3486,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,5,"что "" не надо "" ? не надо меня .",1.827438
3484,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,3,мне уже легче .,1.726226
3482,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,1,"мне показалось , вы меня зовете .",1.698664


In [120]:
submission_name = "linear2_baseline.tsv"

In [122]:
i_test[["context_id", "reply_id"]].to_csv(submission_name, index=False, header=False, sep="\t")

# tmp

In [13]:
pd_valid

NameError: name 'pd_valid' is not defined